In [7]:
import pandas as pd
import xgboost as xgb

from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle

In [26]:
df = pd.read_hdf('newSet.h5')

In [13]:
df = df.loc[df.day<15]

In [14]:
df_ori = df.copy(deep=True)
df_ori.drop('geohash6',axis=1,inplace=True)
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(df_ori)
pickle.dump(scaler, open("scaler.pkl", "wb"))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [9]:
def createScaledAndDf(start,end):
    dfx = df.copy(deep=True)
    dfx.drop('geohash6',axis=1,inplace=True)
    dfx = dfx.loc[(dfx['day']>=start) & (dfx['day']<=end)]
    dfx = dfx.astype(np.float32)
    dfx = dfx.fillna(0)
    scaledx = scaler.transform(dfx)
    print(scaledx.shape)
    return scaledx,dfx
def createTraining(idx,scaledx):
    print(scaledx.shape)
    trainx = scaledx[:idx, :]
    testx = scaledx[idx+1:, :]
    x_trainx, y_trainx = trainx[:, :-1], trainx[:, -1]
    x_testx, y_testx = testx[:, :-1], testx[:, -1]
    
    print(x_trainx.shape, y_trainx.shape, x_testx.shape, y_testx.shape)
    return (x_trainx, y_trainx, x_testx, y_testx)
def pred(x_trainx, y_trainx, x_testx, y_testx,random_s):
    model = XGBRegressor(objective='reg:squarederror',learning_rate=0.035,n_estimators=500,random_state=random_s,max_depth=9
                         ,min_child_weight=5,nthread=-1)
    eval_set = [(x_trainx, y_trainx), (x_testx, y_testx)]
    model.fit(x_trainx, y_trainx, eval_metric="rmse",early_stopping_rounds=20, eval_set=eval_set, verbose=True)
    return model

In [10]:

def pred_sil(x_trainx, y_trainx, x_testx, y_testx,random_s,n_est):
    model = XGBRegressor(objective='reg:squarederror',learning_rate=0.035,n_estimators=n_est,random_state=random_s,max_depth=9,min_child_weight=5,nthread=-1)
    model.fit(x_trainx, y_trainx, eval_metric="rmse", verbose=True)
    return model

In [31]:
import time
def timeStart():
    start = time.time()
    return start
def timeEnd():
    end = time.time()
    print("Total time taken: {} min".format((end - start)/60))

In [34]:
start = timeStart()
import random
days = df['day'].unique()
i = 1
for count, element in enumerate(days, days[0]):
    if count == 11:
        print(df.loc[(df.day >= element) & (df.day <=element+4)].shape)
        print('training for {} to {} days..'.format(element,element+4))
        print('saving to file{}'.format(i))
        print('last file, breaking')
        scaled1,df1 = createScaledAndDf(element,element+5)
        df1.reset_index(0,drop=True,inplace=True)
        train_idx1 = df1.loc[df1['day']<element+4].index[-1]
        rands = random.randint(1,100000)        
        x_train1, y_train1 , x_test1, y_test1 = createTraining(train_idx1,scaled1)
        model1 = pred_sil(x_train1, y_train1, x_test1, y_test1,rands,250)
        pickle.dump(model1, open("model/final{}.model.dat".format(i), "wb"))
        break
    if (count % 2 != 0):
        print(df.loc[(df.day >= element) & (df.day <=element+4)].shape)
        print('training for {} to {} days..'.format(element,element+4))
        print('saving to file{}'.format(i))
        scaled1,df1 = createScaledAndDf(element,element+5)
        df1.reset_index(0,drop=True,inplace=True)
        train_idx1 = df1.loc[df1['day']<element+5].index[-1]
        rands = random.randint(1,100000)        
        x_train1, y_train1 , x_test1, y_test1 = createTraining(train_idx1,scaled1)
        model1 = pred(x_train1, y_train1, x_test1, y_test1,rands)
        y_pred1 = model1.predict(x_test1)
        RMSE1 = sqrt(mean_squared_error(y_test1, y_pred1))
        print("RMS : %.2f%%" % (RMSE1 * 100.0))
        pickle.dump(model1, open("model/final{}.model.dat".format(i), "wb"))
        i+=1
timeEnd()

(347933, 45)
training for 1 to 5 days..
saving to file1
(413429, 44)
(413429, 44)
(347932, 43) (347932,) (65496, 43) (65496,)
[0]	validation_0-rmse:0.410607	validation_1-rmse:0.419654
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:0.396326	validation_1-rmse:0.405056
[2]	validation_0-rmse:0.382547	validation_1-rmse:0.390982
[3]	validation_0-rmse:0.369253	validation_1-rmse:0.377401
[4]	validation_0-rmse:0.356428	validation_1-rmse:0.364304
[5]	validation_0-rmse:0.344054	validation_1-rmse:0.351664
[6]	validation_0-rmse:0.332116	validation_1-rmse:0.339455
[7]	validation_0-rmse:0.320599	validation_1-rmse:0.327693
[8]	validation_0-rmse:0.309488	validation_1-rmse:0.316347
[9]	validation_0-rmse:0.298769	validation_1-rmse:0.305403
[10]	validation_0-rmse:0.288428	validation_1-rmse:0.294846
[11]	validation_0-rmse:0.278453	validation_1-rmse:0.284673
[12]	validation_0

[135]	validation_0-rmse:0.027388	validation_1-rmse:0.028854
[136]	validation_0-rmse:0.027359	validation_1-rmse:0.028842
[137]	validation_0-rmse:0.027331	validation_1-rmse:0.028827
[138]	validation_0-rmse:0.027305	validation_1-rmse:0.028816
[139]	validation_0-rmse:0.02728	validation_1-rmse:0.028806
[140]	validation_0-rmse:0.027252	validation_1-rmse:0.028797
[141]	validation_0-rmse:0.027226	validation_1-rmse:0.028786
[142]	validation_0-rmse:0.027201	validation_1-rmse:0.028778
[143]	validation_0-rmse:0.027178	validation_1-rmse:0.028767
[144]	validation_0-rmse:0.027159	validation_1-rmse:0.028762
[145]	validation_0-rmse:0.027138	validation_1-rmse:0.028754
[146]	validation_0-rmse:0.027119	validation_1-rmse:0.028748
[147]	validation_0-rmse:0.0271	validation_1-rmse:0.028742
[148]	validation_0-rmse:0.027081	validation_1-rmse:0.028735
[149]	validation_0-rmse:0.027063	validation_1-rmse:0.028729
[150]	validation_0-rmse:0.027046	validation_1-rmse:0.028722
[151]	validation_0-rmse:0.027023	validation

[62]	validation_0-rmse:0.053355	validation_1-rmse:0.053795
[63]	validation_0-rmse:0.052011	validation_1-rmse:0.052505
[64]	validation_0-rmse:0.050724	validation_1-rmse:0.051275
[65]	validation_0-rmse:0.049499	validation_1-rmse:0.050103
[66]	validation_0-rmse:0.048325	validation_1-rmse:0.04899
[67]	validation_0-rmse:0.047205	validation_1-rmse:0.04793
[68]	validation_0-rmse:0.046138	validation_1-rmse:0.046921
[69]	validation_0-rmse:0.045121	validation_1-rmse:0.045959
[70]	validation_0-rmse:0.04415	validation_1-rmse:0.045048
[71]	validation_0-rmse:0.043225	validation_1-rmse:0.044181
[72]	validation_0-rmse:0.042345	validation_1-rmse:0.043362
[73]	validation_0-rmse:0.041507	validation_1-rmse:0.042581
[74]	validation_0-rmse:0.040712	validation_1-rmse:0.041844
[75]	validation_0-rmse:0.039953	validation_1-rmse:0.041141
[76]	validation_0-rmse:0.039233	validation_1-rmse:0.040485
[77]	validation_0-rmse:0.03855	validation_1-rmse:0.039862
[78]	validation_0-rmse:0.037903	validation_1-rmse:0.039272
[

[200]	validation_0-rmse:0.025941	validation_1-rmse:0.030427
[201]	validation_0-rmse:0.025929	validation_1-rmse:0.030426
[202]	validation_0-rmse:0.025914	validation_1-rmse:0.030429
[203]	validation_0-rmse:0.025904	validation_1-rmse:0.03043
[204]	validation_0-rmse:0.025885	validation_1-rmse:0.030434
[205]	validation_0-rmse:0.025867	validation_1-rmse:0.030438
[206]	validation_0-rmse:0.02586	validation_1-rmse:0.030438
[207]	validation_0-rmse:0.025847	validation_1-rmse:0.030436
[208]	validation_0-rmse:0.025842	validation_1-rmse:0.030435
[209]	validation_0-rmse:0.025829	validation_1-rmse:0.030434
[210]	validation_0-rmse:0.025817	validation_1-rmse:0.030433
[211]	validation_0-rmse:0.02581	validation_1-rmse:0.030437
[212]	validation_0-rmse:0.025801	validation_1-rmse:0.030438
[213]	validation_0-rmse:0.025792	validation_1-rmse:0.030438
Stopping. Best iteration:
[193]	validation_0-rmse:0.026007	validation_1-rmse:0.030426

RMS : 3.04%
(347080, 45)
training for 4 to 8 days..
saving to file3
(411193,

[119]	validation_0-rmse:0.027687	validation_1-rmse:0.030719
[120]	validation_0-rmse:0.027624	validation_1-rmse:0.030662
[121]	validation_0-rmse:0.027566	validation_1-rmse:0.030609
[122]	validation_0-rmse:0.027507	validation_1-rmse:0.03056
[123]	validation_0-rmse:0.027458	validation_1-rmse:0.030505
[124]	validation_0-rmse:0.027406	validation_1-rmse:0.030455
[125]	validation_0-rmse:0.027358	validation_1-rmse:0.030407
[126]	validation_0-rmse:0.027314	validation_1-rmse:0.030367
[127]	validation_0-rmse:0.027271	validation_1-rmse:0.030328
[128]	validation_0-rmse:0.027233	validation_1-rmse:0.030294
[129]	validation_0-rmse:0.027192	validation_1-rmse:0.030259
[130]	validation_0-rmse:0.027152	validation_1-rmse:0.030228
[131]	validation_0-rmse:0.027113	validation_1-rmse:0.030191
[132]	validation_0-rmse:0.027075	validation_1-rmse:0.030166
[133]	validation_0-rmse:0.027039	validation_1-rmse:0.030135
[134]	validation_0-rmse:0.027004	validation_1-rmse:0.030115
[135]	validation_0-rmse:0.02697	validatio

[57]	validation_0-rmse:0.060889	validation_1-rmse:0.061351
[58]	validation_0-rmse:0.059193	validation_1-rmse:0.059702
[59]	validation_0-rmse:0.057567	validation_1-rmse:0.058123
[60]	validation_0-rmse:0.056009	validation_1-rmse:0.056618
[61]	validation_0-rmse:0.054518	validation_1-rmse:0.05518
[62]	validation_0-rmse:0.05309	validation_1-rmse:0.053801
[63]	validation_0-rmse:0.051725	validation_1-rmse:0.052486
[64]	validation_0-rmse:0.050419	validation_1-rmse:0.051229
[65]	validation_0-rmse:0.04917	validation_1-rmse:0.050031
[66]	validation_0-rmse:0.047977	validation_1-rmse:0.04889
[67]	validation_0-rmse:0.046836	validation_1-rmse:0.047805
[68]	validation_0-rmse:0.045748	validation_1-rmse:0.046771
[69]	validation_0-rmse:0.04471	validation_1-rmse:0.04579
[70]	validation_0-rmse:0.04372	validation_1-rmse:0.04486
[71]	validation_0-rmse:0.042774	validation_1-rmse:0.043971
[72]	validation_0-rmse:0.041876	validation_1-rmse:0.043133
[73]	validation_0-rmse:0.041018	validation_1-rmse:0.042336
[74]	

[195]	validation_0-rmse:0.025035	validation_1-rmse:0.029827
[196]	validation_0-rmse:0.025026	validation_1-rmse:0.029827
[197]	validation_0-rmse:0.025015	validation_1-rmse:0.029828
[198]	validation_0-rmse:0.025003	validation_1-rmse:0.029827
[199]	validation_0-rmse:0.02499	validation_1-rmse:0.029827
[200]	validation_0-rmse:0.024982	validation_1-rmse:0.029826
[201]	validation_0-rmse:0.02497	validation_1-rmse:0.029825
[202]	validation_0-rmse:0.02496	validation_1-rmse:0.029825
[203]	validation_0-rmse:0.02495	validation_1-rmse:0.029825
[204]	validation_0-rmse:0.024943	validation_1-rmse:0.029826
[205]	validation_0-rmse:0.024929	validation_1-rmse:0.029824
[206]	validation_0-rmse:0.024922	validation_1-rmse:0.029821
[207]	validation_0-rmse:0.024909	validation_1-rmse:0.029821
[208]	validation_0-rmse:0.024899	validation_1-rmse:0.029821
[209]	validation_0-rmse:0.024884	validation_1-rmse:0.029817
[210]	validation_0-rmse:0.024874	validation_1-rmse:0.029815
[211]	validation_0-rmse:0.024866	validation_

[51]	validation_0-rmse:0.072455	validation_1-rmse:0.073994
[52]	validation_0-rmse:0.070279	validation_1-rmse:0.071782
[53]	validation_0-rmse:0.068189	validation_1-rmse:0.069661
[54]	validation_0-rmse:0.066182	validation_1-rmse:0.067627
[55]	validation_0-rmse:0.064255	validation_1-rmse:0.065677
[56]	validation_0-rmse:0.062406	validation_1-rmse:0.063807
[57]	validation_0-rmse:0.060632	validation_1-rmse:0.062014
[58]	validation_0-rmse:0.058933	validation_1-rmse:0.060301
[59]	validation_0-rmse:0.057302	validation_1-rmse:0.058658
[60]	validation_0-rmse:0.055742	validation_1-rmse:0.057081
[61]	validation_0-rmse:0.054247	validation_1-rmse:0.055575
[62]	validation_0-rmse:0.052816	validation_1-rmse:0.05414
[63]	validation_0-rmse:0.051447	validation_1-rmse:0.052767
[64]	validation_0-rmse:0.050137	validation_1-rmse:0.051449
[65]	validation_0-rmse:0.048884	validation_1-rmse:0.050196
[66]	validation_0-rmse:0.047685	validation_1-rmse:0.049001
[67]	validation_0-rmse:0.04654	validation_1-rmse:0.047862

[189]	validation_0-rmse:0.024698	validation_1-rmse:0.028034
[190]	validation_0-rmse:0.024694	validation_1-rmse:0.028032
[191]	validation_0-rmse:0.024686	validation_1-rmse:0.028034
[192]	validation_0-rmse:0.024679	validation_1-rmse:0.028035
[193]	validation_0-rmse:0.024669	validation_1-rmse:0.028033
[194]	validation_0-rmse:0.024659	validation_1-rmse:0.028033
[195]	validation_0-rmse:0.024654	validation_1-rmse:0.028033
[196]	validation_0-rmse:0.024645	validation_1-rmse:0.028033
[197]	validation_0-rmse:0.024632	validation_1-rmse:0.02803
[198]	validation_0-rmse:0.024626	validation_1-rmse:0.028029
[199]	validation_0-rmse:0.024615	validation_1-rmse:0.028026
[200]	validation_0-rmse:0.024612	validation_1-rmse:0.028026
[201]	validation_0-rmse:0.024606	validation_1-rmse:0.028025
[202]	validation_0-rmse:0.0246	validation_1-rmse:0.028025
[203]	validation_0-rmse:0.024595	validation_1-rmse:0.028026
[204]	validation_0-rmse:0.024592	validation_1-rmse:0.028026
[205]	validation_0-rmse:0.024585	validation

In [36]:
# print(df.loc[(df.day >= 14) & (df.day <=15)].shape)
# scaled2_1,df2_1 = createScaledAndDf(14,15)
# df2_1.reset_index(0,drop=True,inplace=True)
# train_idx2_1 = df2_1.loc[df2_1['day']<15].index[-1]
# x_train2_1, y_train2_1 , x_test2_1, y_test2_1 = createTraining(train_idx2_1,scaled2_1)

# model1 = pickle.load(open("model/final1.model.dat", "rb"))
# model2 = pickle.load(open("model/final2.model.dat", "rb"))
# model3 = pickle.load(open("model/final3.model.dat", "rb"))
# model4 = pickle.load(open("model/final4.model.dat", "rb"))
# model5 = pickle.load(open("model/final5.model.dat", "rb"))
# model6 = pickle.load(open("model/final6.model.dat", "rb"))

# y_test_final = y_test2_1
# x_test_final = x_test2_1
# y_pred_final = (model1.predict(x_test_final) + model2.predict(x_test_final)+model3.predict(x_test_final)+
#                 model4.predict(x_test_final) + model5.predict(x_test_final))/5
# # RMS : 2.73%
# RMSEF = sqrt(mean_squared_error(y_test_final, y_pred_final))

# print("RMS : %.2f%%" % (RMSEF * 100.0))

(138008, 45)
(138008, 44)
(138008, 44)
(68715, 43) (68715,) (69292, 43) (69292,)
RMS : 2.96%
